# Data Structures and Algorithms in Python - Ch.8,9,11: Tree Exercises
### AJ Zerouali, 2023/10/09

## 0) Introduction

This notebook contains some exercises on trees, and focus on binary search trees and heaps.

**References:**

- Chapters 8, 9, and 11 of "Data structures and algorithms in Python", by Goodrich, Tamassia and Goldwasser (primary). 
- Section 14 of "Python for Data Structures, Algorithms, and Interviews!" by Jose Portilla.


## Exercise 1: Verifying that a binary tree is a BST

Link: https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/06-Trees/Trees%20Interview%20Problems/01-Binary-Search-Tree-Check/01-Binary%20Search%20Tree%20Check.ipynb

Given a binary tree, check whether it’s a binary search tree or not.

**Comment:** This is a classic interview question on BSTs.


### 1) My Solution

Assuming we are given a binary tree, checking whether or not it is a BST boils down to checking whether it preserves the binary search tree property, i.e. that an inorder traversal will give keys in increasing order. My strategy is to start from the root of the tree, produce a list of keys with the inorder traversal, and then check that all keys to the left of any node are lower than the key at that node.

My implementation below is tailored to the *BSTMap* and *HeapPriorityQ* classes in the *Code.Trees* submodule.

In [1]:
from Code.Trees.binary_search_tree_ajz import BSTMap
from Code.Trees.heap_priority_queue import HeapPriorityQ

In [ ]:
def inorder_traversal_list(T, position=None, key_list = None):
    
    if key_list is None:
        key_list = []
    
    if isinstance(T, HeapPriorityQ):
        if position is None:
            position = 0
        # Traverse left subtree
        if T._has_left(position):
            inorder_traversal_list(T, T._left_child(position), key_list)
        # Print element at current position
        #print(f"({T._data[position]._key}, {T._data[position]._value})")
        key_list.append(T._data[position]._key)
        # Traverse right subtree
        if T._has_right(position):
            inorder_traversal_list(T, T._right_child(position), key_list)
            
    elif isinstance(T, BSTMap):
        if position is None:
            position = T.root()
        # Traverse left subtree
        if T.left(position) is not None:
            inorder_traversal_list(T, T.left(position), key_list)
        # Print element at current position
        #print(f"({position.element()._key}, {position.element()._value})")
        key_list.append(position.key())
        # Traverse right subtree
        if T.right(position) is not None:
            inorder_traversal_list(T, T.right(position), key_list)
    
    return key_list

def check_bst(T):
    key_list = inorder_traversal_list(T)
    
    for i in range(1,len(key_list)):
        for j in range(0,i):
            if key_list[j]>=key_list[i]:
                print("T is not a binary search tree")
                return False
    
    print("T is a binary search tree")
    return True

#### Example

In [2]:
L = [(4,"C"), (5, "A"), (6,"Z"), (15,"K"), (9, "F"), 
      (7, "Q"), (20,"B"), (16,"X"), (25, "J"), (14,"E"), 
      (12, "H"), (11, "S"), (13, "W"), 
     ]

In [ ]:
# Build heap of Fig.9.1 in GTG13. Use same nodes in a BST
heap1 = HeapPriorityQ()
bst1 = BSTMap()
for e in L:
    heap1.add(e[0], e[1])
    bst1[e[0]] = e[1]


In [17]:
def inorder_traversal_print(T, position=None):
    
    if isinstance(T, HeapPriorityQ):
        if position is None:
            position = 0
        # Traverse left subtree
        if T._has_left(position):
            inorder_traversal_print(T, T._left_child(position))
        # Print element at current position
        print(f"({T._data[position]._key}, {T._data[position]._value})")
        # Traverse right subtree
        if T._has_right(position):
            inorder_traversal_print(T, T._right_child(position))
            
    elif isinstance(T, BSTMap):
        if position is None:
            position = T.root()
        # Traverse left subtree
        if T.left(position) is not None:
            inorder_traversal_print(T, T.left(position))
        # Print element at current position
        print(f"({position.element()._key}, {position.element()._value})")
        # Traverse right subtree
        if T.right(position) is not None:
            inorder_traversal_print(T, T.right(position))

In [18]:
inorder_traversal_print(bst1)

(4, C)
(5, A)
(6, Z)
(7, Q)
(9, F)
(11, S)
(12, H)
(13, W)
(14, E)
(15, K)
(16, X)
(20, B)
(25, J)


In [19]:
inorder_traversal_print(heap1)

(16, X)
(15, K)
(25, J)
(5, A)
(14, E)
(9, F)
(12, H)
(4, C)
(11, S)
(7, Q)
(13, W)
(6, Z)
(20, B)


In [ ]:
# "Display" underlying array of heap
## The tree can be drawn from this list thanks to complete tree property
## and heap-order poperty
heap1._data

In [28]:
check_bst(heap1)

T is not a binary search tree


False

If we execute our *check_bst()* function we get the following.

In [29]:
check_bst(bst1)

T is a binary search tree


True

## Exercise 2: Printing nodes in every level of a binary tree

Link: https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/06-Trees/Trees%20Interview%20Problems/02-Tree-Level-Order-Print/02-Tree%20Level%20Order%20Print.ipynb

Given a binary tree of integers, print it in level order. The output will contain space between the numbers in the same level, and new line between different levels.

For example, if the tree is as follows:

             1
           /   \
          2     3
         /     / \
        4     5   6       
then the output should be

        1 
        2 3 
        4 5 6

### 1) My solution:

Again, the first reflex is to traverse the binary tree in order. The action at each node will be to add the current key to a string, which contains the keys of all nodes at a given level. This inorder traversal will require us to keep track of the level starting from the root, and I would use a dictionary to store these strings.

In [1]:
from Code.Trees.binary_search_tree_ajz import BSTMap
from Code.Trees.heap_priority_queue import HeapPriorityQ

In [56]:
def inorder_traversal_level(T, position=None, level = None, lvl_dict = None):
    
    if T.is_empty():
        raise ValueError("Tree is empty")
        
    if isinstance(T,HeapPriorityQ):
        if position is None:
            position = 0
            level = 0
            lvl_dict = {}
        
        if T._has_left(position):
            #print(f"Visiting left subtree of ({T._data[position]._key}, {T._data[position]._value})")
            inorder_traversal_level(T, T._left_child(position), level+1, lvl_dict)
            
        if level not in lvl_dict:
            lvl_dict[level] = f"L{level}: "
        lvl_dict[level] += str(T._data[position]._key)+" "
        
        if T._has_right(position):
            #print(f"Visiting right subtree of ({T._data[position]._key}, {T._data[position]._value})")
            inorder_traversal_level(T, T._right_child(position), level+1, lvl_dict)
    
    elif isinstance(T,BSTMap):
        
        if position is None:
            position = T.root()
            level = 0
            lvl_dict = {}
        
        if T.left(position) is not None:
            inorder_traversal_level(T, T.left(position), level+1, lvl_dict)
            
        if level not in lvl_dict:
            lvl_dict[level] = f"L{level}: "
        lvl_dict[level] += str(position.key())+" "
        
        if T.right(position) is not None:
            inorder_traversal_level(T, T.right(position), level+1, lvl_dict)
    
    return lvl_dict

In [32]:
def BT_print_levels(T):
    
    lvl_dict = inorder_traversal_level(T)
    
    levels = sorted(list(lvl_dict.keys()))
    
    for l in levels:
        print(lvl_dict[l])

#### Example

We use the *BSTMap* and *HeapPriorityQ* of the previous example. For visualization though, it will be useful to 

In [47]:
def inorder_traversal_print(T, position=None):
    
    if isinstance(T, HeapPriorityQ):
        if position is None:
            position = 0
        # Traverse left subtree
        if T._has_left(position):
            inorder_traversal_print(T, T._left_child(position))
        # Print element at current position
        if T._parent(position)<0:
            parent_key = "None"
            parent_value = "None"
        else:
            parent_key = T._data[T._parent(position)]._key
            parent_value = T._data[T._parent(position)]._value
        print(f"({T._data[position]._key}, {T._data[position]._value}), position = {position}, Parent = ({parent_key}, {parent_value})")
        # Traverse right subtree
        if T._has_right(position):
            inorder_traversal_print(T, T._right_child(position))
            
    elif isinstance(T, BSTMap):
        if position is None:
            position = T.root()
        # Traverse left subtree
        if T.left(position) is not None:
            inorder_traversal_print(T, T.left(position))
        
        # Print element at current position and its parent
        if T.parent(position) is None:
            parent_key = "None"
            parent_value = "None"
        else:
            parent_key = T.parent(position).key()
            parent_value = T.parent(position).value()
        print(f"({position.element()._key}, {position.element()._value}), Parent = ({parent_key}, {parent_value})")
        
        # Traverse right subtree
        if T.right(position) is not None:
            inorder_traversal_print(T, T.right(position))

In [33]:
L = [(4,"C"), (5, "A"), (6,"Z"), (15,"K"), (9, "F"), 
      (7, "Q"), (20,"B"), (16,"X"), (25, "J"), (14,"E"), 
      (12, "H"), (11, "S"), (13, "W"), 
     ]

In [34]:
# Build heap of Fig.9.1 in GTG13. Use same nodes in a BST
heap1 = HeapPriorityQ()
bst1 = BSTMap()
for e in L:
    heap1.add(e[0], e[1])
    bst1[e[0]] = e[1]


This gives us the following trees

In [48]:
inorder_traversal_print(bst1)

(4, C), Parent = (None, None)
(5, A), Parent = (4, C)
(6, Z), Parent = (5, A)
(7, Q), Parent = (9, F)
(9, F), Parent = (15, K)
(11, S), Parent = (12, H)
(12, H), Parent = (14, E)
(13, W), Parent = (12, H)
(14, E), Parent = (9, F)
(15, K), Parent = (6, Z)
(16, X), Parent = (20, B)
(20, B), Parent = (15, K)
(25, J), Parent = (20, B)


In [49]:
inorder_traversal_print(heap1)

(16, X), position = 7, Parent = (15, K)
(15, K), position = 3, Parent = (5, A)
(25, J), position = 8, Parent = (15, K)
(5, A), position = 1, Parent = (4, C)
(14, E), position = 9, Parent = (9, F)
(9, F), position = 4, Parent = (5, A)
(12, H), position = 10, Parent = (9, F)
(4, C), position = 0, Parent = (None, None)
(11, S), position = 11, Parent = (7, Q)
(7, Q), position = 5, Parent = (6, Z)
(13, W), position = 12, Parent = (7, Q)
(6, Z), position = 2, Parent = (4, C)
(20, B), position = 6, Parent = (6, Z)


Now printing the contents of every level gives us the following:

In [35]:
BT_print_levels(bst1)

L0: 4 
L1: 5 
L2: 6 
L3: 15 
L4: 9 20 
L5: 7 14 16 25 
L6: 12 
L7: 11 13 


In [63]:
BT_print_levels(heap1)

L0: 4 
L1: 5 6 
L2: 15 9 7 20 
L3: 16 25 14 12 11 13 


### 2) Portilla's solution

Link: https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/06-Trees/Trees%20Interview%20Problems/02-Tree-Level-Order-Print/02-Tree%20Level%20Order%20Print%20-%20SOLUTION.ipynb

The idea is to use breadth-first search

## Exercise 3: Trimming a BST

Link: https://github.com/jmportilla/Python-for-Algorithms--Data-Structures--and-Interviews/blob/master/06-Trees/Trees%20Interview%20Problems/03-Trim-a-Binary-Search-Tree/03-Trim%20a%20Binary%20Search%20Tree%20.ipynb

Given the root of a binary search tree and 2 numbers min and max, trim the tree such that all the numbers in the new tree are between min and max (inclusive). The resulting tree should still be a valid binary search tree. 

**Example:** If we get this tree as input: 

              8
            /   \
           3     10
          / \     \
         1  6     14     
           / \   /
          4  7  13

and if *min=5* and *max=13*, then the resulting trimmed BST should be:
    
             8
           /  \
          6    10
           \    \
           7    13     

### 1) My solution:

For this problem, we need to remove the nodes from the lowest keys to the largest keys, and always start with the nodes at the highest levels. This is essentially a postorder traversal of the binary tree, where after exploring the left and right subtrees, the performed action is checking whether or not the key of the current node is within the desired interval. If not, then we delete the node.

In [8]:
from Code.Trees.binary_search_tree_ajz import BSTMap
from Code.Trees.heap_priority_queue import HeapPriorityQ
from copy import deepcopy

In [5]:
def inorder_traversal_print(T, position=None):
    
    if isinstance(T, HeapPriorityQ):
        if position is None:
            position = 0
        # Traverse left subtree
        if T._has_left(position):
            inorder_traversal_print(T, T._left_child(position))
        # Print element at current position
        if T._parent(position)<0:
            parent_key = "None"
            parent_value = "None"
        else:
            parent_key = T._data[T._parent(position)]._key
            parent_value = T._data[T._parent(position)]._value
        print(f"({T._data[position]._key}, {T._data[position]._value}), position = {position}, Parent = ({parent_key}, {parent_value})")
        # Traverse right subtree
        if T._has_right(position):
            inorder_traversal_print(T, T._right_child(position))
            
    elif isinstance(T, BSTMap):
        if position is None:
            position = T.root()
        # Traverse left subtree
        if T.left(position) is not None:
            inorder_traversal_print(T, T.left(position))
        
        # Print element at current position and its parent
        if T.parent(position) is None:
            parent_key = "None"
            parent_value = "None"
        else:
            parent_key = T.parent(position).key()
            parent_value = T.parent(position).value()
        print(f"({position.element()._key}, {position.element()._value}), Parent = ({parent_key}, {parent_value})")
        
        # Traverse right subtree
        if T.right(position) is not None:
            inorder_traversal_print(T, T.right(position))

In [18]:
def TrimBST(T, min_key, max_key, position = None):
    
    if T.is_empty():
        raise ValueError("Tree is empty")
    
    if position is None:
        position = T.root()
    
    if T.left(position) is not None:
        TrimBST(T, min_key, max_key, T.left(position))
    
    if T.right(position) is not None:
        TrimBST(T, min_key, max_key, T.right(position))
    
    if (position.key()<min_key) or (position.key()>max_key):
        #print(f"Removing position ({position.key()}, {position.value()})")
        T.delete(position)
        #del T[position.key()]
    
    


#### Example 1

In [3]:
L = [(4,"C"), (5, "A"), (6,"Z"), (15,"K"), (9, "F"), 
      (7, "Q"), (20,"B"), (16,"X"), (25, "J"), (14,"E"), 
      (12, "H"), (11, "S"), (13, "W"), 
     ]

In [4]:
# Build heap of Fig.9.1 in GTG13. Use same nodes in a BST
heap1 = HeapPriorityQ()
bst1 = BSTMap()
for e in L:
    heap1.add(e[0], e[1])
    bst1[e[0]] = e[1]


In [6]:
inorder_traversal_print(bst1)

(4, C), Parent = (None, None)
(5, A), Parent = (4, C)
(6, Z), Parent = (5, A)
(7, Q), Parent = (9, F)
(9, F), Parent = (15, K)
(11, S), Parent = (12, H)
(12, H), Parent = (14, E)
(13, W), Parent = (12, H)
(14, E), Parent = (9, F)
(15, K), Parent = (6, Z)
(16, X), Parent = (20, B)
(20, B), Parent = (15, K)
(25, J), Parent = (20, B)


In [16]:
bst2 = deepcopy(bst1)

In [19]:
TrimBST(bst2, min_key = 6, max_key = 15)

In [20]:
inorder_traversal_print(bst2)

(6, Z), Parent = (None, None)
(7, Q), Parent = (9, F)
(9, F), Parent = (15, K)
(11, S), Parent = (12, H)
(12, H), Parent = (14, E)
(13, W), Parent = (12, H)
(14, E), Parent = (9, F)
(15, K), Parent = (6, Z)


#### Example 2

In [29]:
del bst_ex

In [30]:
E = [ (8, "Q"), 
     (3, "A"), (1,"C"), (6,"Z"), (4,"K"), (7, "F"), 
      (10,"B"), (14,"X"), (13, "W"), 
     ]
bst_ex = BSTMap()
for e in E:
    bst_ex[e[0]] = e[1]
bst_ex2 = deepcopy(bst_ex)

In [31]:
inorder_traversal_print(bst_ex)

(1, C), Parent = (3, A)
(3, A), Parent = (8, Q)
(4, K), Parent = (6, Z)
(6, Z), Parent = (3, A)
(7, F), Parent = (6, Z)
(8, Q), Parent = (None, None)
(10, B), Parent = (8, Q)
(13, W), Parent = (14, X)
(14, X), Parent = (10, B)


In [33]:
TrimBST(bst_ex2, min_key = 5, max_key = 13)

In [34]:
inorder_traversal_print(bst_ex2)

(6, Z), Parent = (8, Q)
(7, F), Parent = (6, Z)
(8, Q), Parent = (None, None)
(10, B), Parent = (8, Q)
(13, W), Parent = (10, B)


# Scrap